## 1. Introduction
In this project I will generate an interactive dashboard of visualizations that will allow us to explore the dataset results of a university course's surveys.

At the end of each semester, the students are presented with a survey for each one of their courses that asks them about their learning experience and the performance of their teacher.<br>
The survey is in Spanish and has 16 questions, but I will be focusing on only three of them.
Two questions of multiple choice type, and one question that asks them for general comments about the teacher.

The idea for the exploration is to try to identify certain types of words or patterns used in the comments by the students depending on the type of response they gave towards the first and second multiple choice questions of the survey.

The dashboard and all its texts will be in spanish but I will provide a translation for them.

## 2. Visualization Technique

I will create a dashboard with four graphs:
- A bar plot with the frequency counts of each category response of the first question as a percentage.
- A pie chart with the frequency counts of each category response of the second question as a percentage.
- A word cloud image with the words most used in the comments question displayed from bigger to smaller according to their frequency ocurrence.
- A scatter dot plot with the total number of words used in the comments on each course.

Aditionally, we will add five interactive widgets to manipulate the dataset and presentation:
- Two checkboxes to enable/disable the filter for the first and second question's dataset. They can be used together to apply both filters at the same time.
- Two sliders to select the response category that we want the mean of responses to lean towards from the first and second questions. 
- A third slider to select the minimum word length for the word cloud and scatter dot graphs.

The bar plot will help us to easily compare categorical data by representing values with rectangular bars. It helps us to quickly identify trends, patterns, and differences between categories, making it useful for analyzing the survey results. Also will allow us to easily identify the skewness of the responses.

The pie chart will show us the proportional distribution of all the question responses by dividing it into slices. It is useful for quickly understanding relative contributions of different categories in the survey responses.

The word cloud image can be used to represent text data by displaying words in varying sizes based on their frequency or importance. It will help us identifying key themes, trends, or dominant topics in the survey responses.

The scatter dot plot technique will be used to display individual data points of the dataset to show distribution, frequency, and relationships between the variables. In our case will help us identifying patterns, outliers, and correlations in the survey results.

## 3. Visualization Library
The libraries used for this project integrate very well with Jupyter and you can easily install them all with PiP or Conda.
The list of libraries is in the "requirements.txt" file.

<u>Visualization libraries:</u><br>

The main visualization library I will use is Matplotlib which is a powerful open-source and open-source Python visualization library that enables us to create a wide range of static, animated, and interactive plots, including line charts, bar plots, scatter plots, and histograms, for data analysis and presentation.<br>
It was created by John D. Hunter in 2003 and you can find the API reference documentation, tutorials and example guides at the [official site](https://matplotlib.org/stable/api/index.html).

Next, I will use Wordcloud which is a procedural library that generates word clouds from text data, where word size represents frequency or importance, and uses Matplotlib for rendering the visualizations.<br>
The usage of this library is pretty simple and straightforward.<br>
It was created by Andreas Müller in 2013 and you can find the API reference documentation and many examples of it's usage at his [website](https://amueller.github.io/word_cloud/auto_examples/index.html#example-gallery).

Finally, I will use the IPython and Ipywidgets libraries, created by Fernando Pérez in 2001 and 2014, and will enable us to use interactive visualizations and user interface controls in Jupyter notebooks by providing widgets like sliders, buttons, and dropdowns to dynamically manipulate data and visual outputs.

<u>Data processing libraries:</u><br>

The main data processing library I am using is Pandas, along with its excel optional dependency that allows us to read and write Microsoft Excel files. This is the [official site](https://pandas.pydata.org/docs/reference/index.html) of Pandas with the API documentation, user guides and release notes.<br>

For data processing I am using Spacy which is a natural language processing (NLP) library created by Matthew Honnibal and is designed for advanced text analysis, providing the tools for tokenization and lemmatization in many languages, including Spanish.<br> You can find the API documentation, library and model arquitectures as well as examples in the [official site](https://spacy.io/api/doc).
The Spanish language model package used is "es_core_news_sm".

## 4. Demonstration

Importing all neccesary dependencies.<br>
The downloading of the language model needs to be done only once. On subsequent runs it can be commented off.

In [2]:
import numpy as np
import json
import textwrap
import re
import pandas as pd
import matplotlib.pyplot as plt, matplotlib.ticker as ticker
import spacy
spacy.cli.download("es_core_news_sm")
from PIL import Image
from wordcloud import WordCloud
from ipywidgets import widgets, Checkbox, IntSlider, SelectionSlider
from IPython.display import display, clear_output

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 43.0 MB/s eta 0:00:00 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


This is loading the initial dataset which is an excel document consisting of just one table with the question's answer's categories as columns and the course's ID's as rows.<br>
The responses per question category are integers of the frequencies of the responses.<br>
Row 0 contains the name of the questions.<br>
Row 1 contains the question's categories.

In [3]:
data = pd.read_excel('assets/ResultadosFinales Public.xlsx', header=[0, 1], index_col=[0, 1]) #The first two rows are the headers.
data.head()

InfoCurso           InfoCurso                      \
ID del Curso Total de Alumnos Total de Respuestas   
0 32896-1                  27                  15   
1 32896-10                 31                  17   
2 32896-11                 30                  16   
3 32896-2                  32                  18   
4 32896-3                  34                  13   

InfoCurso    1. ¿En la primera semana de clase, el profesor ha explicado el contenido de su sílabo?  \
ID del Curso                                                                                     Sí   
0 32896-1                                                  12.0                                       
1 32896-10                                                 14.0                                       
2 32896-11                                                 15.0                                       
3 32896-2                                                  17.0                                       
4 32896-3                                                  13.0                                       

InfoCurso                             \
ID del Curso   No No sabe / No opina   
0 32896-1     0.0                3.0   
1 32896-10    1.0                1.0   
2 32896-11    0.0                1.0   
3 32896-2     0.0                1.0   
4 32896-3     0.0                0.0   

InfoCurso    2. ¿El profesor ha proporcionado oportunamente a los estudiantes recursos (documentos, videos, enlaces de internet, presentaciones, etc.) durante el desarrollo de la asignatura?  \
ID del Curso                                                                                                                                                                           Siempre   
0 32896-1                                                  12.0                                                                                                                                  
1 32896-10                                                  7.0                                                                                                                                  
2 32896-11                                                 16.0                                                                                                                                  
3 32896-2                                                  11.0                                                                                                                                  
4 32896-3                                                  12.0                                                                                                                                  

InfoCurso                                                 ...  \
ID del Curso Casi siempre Algunas veces Casi nunca Nunca  ...   
0 32896-1             3.0           0.0        0.0   0.0  ...   
1 32896-10            5.0           3.0        1.0   0.0  ...   
2 32896-11            0.0           0.0        0.0   0.0  ...   
3 32896-2             3.0           4.0        0.0   0.0  ...   
4 32896-3             1.0           0.0        0.0   0.0  ...   

InfoCurso    14. ¿La forma de enseñar del jefe de laboratorio, taller o centro de cómputo te ha permitido aprender con facilidad?  \
ID del Curso                                                                                                         Casi siempre   
0 32896-1                                                   NaN                                                                     
1 32896-10                                                  NaN                                                                     
2 32896-11                                                  NaN                                                                     
3 32896-2                                                   NaN                                                                     
4 32896-3   

### Cleaning and filtering the dataset.

First grabbing the question's texts from the dataset and using the Textwrap library to easily insert a new line every specific ammount of characters. The parameter width equals number of characters.<br>
Then removing the top header with droplevel(0) since I won't need it.

In [4]:
q1_wrapped_title = "\n".join(textwrap.wrap(data.columns[23][0][3:], width=50)) #Question 1 - q1
q2_wrapped_title = "\n".join(textwrap.wrap(data.columns[49][0][4:], width=50)) #Question 2 - q2
data.columns = data.columns.droplevel(0) #Removing top header with the question's sentence

Removing the rows with no student answers.<br>
And finally I reduce the dataset to just the columns needed.

In [5]:
data = data[data['Total de Respuestas']>0] #Filtering the rows that don't have any survey answers
data = data.iloc[:, [23,24,25,26,27,48,49,50,51,52,53]] #From 23-27 are q1 answers, 48-52 are q2 answers, and 53 are the comments

Here we are calculating the categorical mean of the question's responses by multiplying each question's responses times a category value and dividing by total answers.<br>
Then rounding to the most proximate category.

The resulting mean tells us which category is the average for this course according to the student responses.

In [6]:
data['q1_mean'] = (data.iloc[:, 0:5].mul([0,1,2,3,4], axis=1).sum(axis=1) / data.iloc[:, 0:5].sum(axis=1)).round() 
data['q2_mean'] = (data.iloc[:, 5:10].mul([0,1,2,3,4], axis=1).sum(axis=1) / data.iloc[:, 5:10].sum(axis=1)).round()
data.head()

,ID del Curso,Siempre,Casi siempre,Algunas veces,Casi nunca,Nunca,Muy buena,Buena,Regular,Mala,Muy mala,Comentario,q1_mean,q2_mean
0,32896-1,7.0,4.0,3.0,1.0,0.0,10.0,4.0,1.0,0.0,0.0,#1. Nada\n#2. En que trate de hacer los temas ...,1.0,0.0
1,32896-10,3.0,4.0,5.0,1.0,3.0,3.0,4.0,4.0,1.0,1.0,"#1. En brindar las rubricad adecuadas, silabos...",2.0,1.0
2,32896-11,14.0,1.0,1.0,0.0,0.0,14.0,2.0,0.0,0.0,0.0,"#1. Esta acordé con su enseñanza, muy didáctic...",0.0,0.0
3,32896-2,5.0,9.0,2.0,1.0,0.0,9.0,6.0,3.0,0.0,0.0,"#1. el metodo de ensayo, un grupo de alumnos t...",1.0,1.0
4,32896-3,10.0,2.0,1.0,0.0,0.0,11.0,2.0,0.0,0.0,0.0,#1. En terminar la clase a tiempo\n#2. Nada\n#...,0.0,0.0


Here I am loading a json file with the lists of articles, prepositions, pronouns, conjunctions, symbols and extras. In this case the extras key is empty but I could easily update the json file adding to it more words I'd like to filter.<br>
We then combine all stop words in the json file into one single list.

In [7]:
with open("assets/stop_words_spanish_no_verbs.json", "r", encoding="utf-8") as file:
    stop_words_dic = json.load(file)
stop_words = sum(stop_words_dic.values(), []) #Combining all the lists

Here I loop through all the comments in the dataset and apply transformations and filters.

For each row, I convert the initial comment which is just one big string into a list of words by using spacy, which performs both the tokenization and lemmatization operations.<br>
Then I use the regex library to split the words that may be conjoined by symbols (ie: open-source, decisive[making], etc) with re.escape()<br>
Finally, I check if the finished word set doesn't contain a stop word or a number.

At the end I am also creating a new column with the count of words to be used later in the scatter dot plot.

Running nlp(str(comment)) should take about 30 seconds. 


<u>Note about using spacy:</u><br>
In some cases, depending on the language, spacy converts certain words that are composed of multiple types of words (verbs, pronouns, articles, etc) into one string with a space separating the split words.<br>
For example, in spanish the word 'cofírmese' is transformed into 'confirmar él'<br>

In [8]:
nlp = spacy.load("es_core_news_sm") #Creating a new spacy instance with the spanish package

comments = data.iloc[:,10] #Grabbing the comments column
regex_pattern_separator = "[" + re.escape("".join(stop_words_dic['símbolos'])) + "]" #Pattern of delimiters to split words by
filtered_lemmatized_list = [] #The clean and finished list of lists of all the comments turned into tokenized lemmas filtered

for comment in comments:
    tokenized = nlp(str(comment)) #Tokenizes and lemmatizes the comment
    lemmas = [token.lemma_ for token in tokenized if 'http' not in token.lemma_ and '--' not in token.lemma_] #Getting the lemmas in string form and filtering urls and lines
    lemmas = [part for word in lemmas for part in re.split(regex_pattern_separator, word) if part] #Split words that are conjoined with punctuation symbols, and spaces in some cases 
    lemmas = [word for word in lemmas if word not in stop_words and not word.isdigit()] #Remove words that are stop_words or numbers
    filtered_lemmatized_list.append(lemmas)
data['Comments_lemmatized'] = filtered_lemmatized_list

data['number_of_words'] = data['Comments_lemmatized'].apply(lambda word_list: len(word_list)) #Setting the number of words column
data.head()

,ID del Curso,Siempre,Casi siempre,Algunas veces,Casi nunca,Nunca,Muy buena,Buena,Regular,Mala,Muy mala,Comentario,q1_mean,q2_mean,Comments_lemmatized,number_of_words
0,32896-1,7.0,4.0,3.0,1.0,0.0,10.0,4.0,1.0,0.0,0.0,#1. Nada\n#2. En que trate de hacer los temas ...,1.0,0.0,"[nada, tratar, hacer, tema, más, sencillo, ent...",83
1,32896-10,3.0,4.0,5.0,1.0,3.0,3.0,4.0,4.0,1.0,1.0,"#1. En brindar las rubricad adecuadas, silabos...",2.0,1.0,"[brindar, rubricad, adecuado, silabo, adecuado...",118
2,32896-11,14.0,1.0,1.0,0.0,0.0,14.0,2.0,0.0,0.0,0.0,"#1. Esta acordé con su enseñanza, muy didáctic...",0.0,0.0,"[este, acordar, su, enseñanza, mucho, didáctic...",46
3,32896-2,5.0,9.0,2.0,1.0,0.0,9.0,6.0,3.0,0.0,0.0,"#1. el metodo de ensayo, un grupo de alumnos t...",1.0,1.0,"[metodo, ensayo, uno, grupo, alumno, tener, me...",56
4,32896-3,10.0,2.0,1.0,0.0,0.0,11.0,2.0,0.0,0.0,0.0,#1. En terminar la clase a tiempo\n#2. Nada\n#...,0.0,0.0,"[terminar, clase, tiempo, nada, mm, parecer, c...",20


### Presentation

The Wordcloud library allows us to use an image to display the results of the word cloud.<br>
For this we need to use a black and white image.<br>
In this project I am using an already black and white image but the code bellow will work with any type of image by first converting it to greyscale and then clamping the values to pure white or pure black.

This is the only usage of the Pillow library, to open the image with Image.open

In [9]:
words_mask = np.array(Image.open("assets/silhouette.jpg").convert("L")) #Loading a black and white image with convert("L") to transform it to greyscale
words_mask = np.where(words_mask > 128, 255, 0) #Clamping the greyscale values to 0 or 255

Here I am creating a dictionary with the categories of each question's answer's as keys and a number as value, so that we can easily index them with the widget slider's values.

In [10]:
category_values = [0,1,2,3,4] #Values for the question's answer's categories.
q1_keys = list(data.iloc[:, [0,1,2,3,4]].columns) #Question 1 categories (columns 0 to 4)
q2_keys = list(data.iloc[:, [5,6,7,8,9]].columns) #Question 1 categories (columns 5 to 9)
q1_options = {k: v for k, v in zip(q1_keys, category_values)}
q2_options = {k: v for k, v in zip(q2_keys, category_values)}

Here I am creating the five widgets that will allow us to make the dashboard visualization dynamic and explore much more effectively the data.

We set the initial value of the widgets with the parameter <b>value</b><br>
We can configure the appearance of the widgets with the <b>style</b> and <b>layout</b> attributes.

In [11]:
widget_q1_checkbox = Checkbox(value=False, description="Habilitar filtro")
widget_q2_checkbox = Checkbox(value=False, description="Habilitar filtro")
widget_q1_slider = SelectionSlider(value=q1_keys[2], options=q1_keys, description="Pregunta 1 media:", style={'description_width': '200px'}, layout=widgets.Layout(width='500px'))
widget_q2_slider = SelectionSlider(value=q2_keys[2], options=q2_keys, description="Pregunta 2 media:", style={'description_width': '200px'}, layout=widgets.Layout(width='500px'))
widget_letter_slider = IntSlider(value=3, min=1, max=30, step=1, description='Mínimo de letras por palabra:', style={'description_width': '200px'}, layout=widgets.Layout(width='1000px'))

Here we are organizing the widgets position using a collection of horizontal and vertical boxes.<br>
Again, we can further configure the layout with the <b>layout</b> attribute.

In [12]:
vbox_q1 = widgets.VBox([widget_q1_checkbox,widget_q1_slider], layout=widgets.Layout(margin='20px'))
vbox_q2 = widgets.VBox([widget_q2_checkbox,widget_q2_slider], layout=widgets.Layout(margin='20px'))
hbox_q = widgets.HBox([vbox_q1, vbox_q2])
final_box = widgets.VBox([hbox_q,widget_letter_slider], layout=widgets.Layout(margin='0px 20px 20px 20px')) #Final box that holds all previous boxes

This is the principal function that creates the four plots of the dashboard and will get updated on every change of our widgets.<br>


In [13]:
def Draw_plots(q1_checkbox, q1_slider, q2_checkbox, q2_slider, letter_slider):
    with output:
        clear_output(wait=True) #Necessary to clear the previous output before redrawing

        fig, axes = plt.subplots(2, 2, figsize=(15, 10)) #Creating a grid of plots of 2 rows and 2 columns. Figzize determines the width and height of the figure.
        plt.subplots_adjust(hspace=0.3) #Spacing between plots
        axes = axes.flatten() #Makes it easier to access the subplots as a 1D array

        # FILTERING DATA
        filtered_data = data
        if q1_checkbox:
            filtered_data = filtered_data[filtered_data['q1_mean']==q1_options[q1_slider]] #Applying filter
        if q2_checkbox:
            filtered_data = filtered_data[filtered_data['q2_mean']==q2_options[q2_slider]] #Applying filter

        if not filtered_data.empty: 
            # BAR PLOT - QUESTION 1
            data_q1 = filtered_data.iloc[:, [0,1,2,3,4]].sum() #Question 1 answer categories total
            data_q1 = data_q1/data_q1.sum()*100 #Converting to percentage          
            axes[0].bar(data_q1.index, data_q1.values, color='blue', alpha= 1 if q1_checkbox else 0.2) #Creating the bar plot
            axes[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f"{int(x)}%"))
            axes[0].text(0.5, 1.1, q1_wrapped_title, ha='center', fontsize=12, transform=axes[0].transAxes) #Creating a text at position x=0.5, y=1.1 starting at bottom-left on the axes[0] coordinate system
            # PIE CHART - QUESTION 2
            data_q2 = filtered_data.iloc[:, [5,6,7,8,9]].sum() #Question 2 answer categories total
            data_q2 = data_q2/data_q2.sum()*100 #Converting to percentage
            wedges, _, _ = axes[1].pie(data_q2.values, labels=data_q2.index, autopct='%1.1f%%') #Creating the pie chart
            [wedge.set_alpha(1 if q2_checkbox else 0.2) for wedge in wedges] #Setting opacity of the wedges
            axes[1].text(0.5, 1.1, q2_wrapped_title, ha='center', fontsize=12, transform=axes[1].transAxes) #Creating a text at position x=0.5, y=1.1 starting at bottom-left on the axes[1] coordinate system
            # WORD CLOUD
            words_all = filtered_data['Comments_lemmatized'].apply(lambda word_list: [word for word in word_list if len(word) >= letter_slider]) #Removing words smaller than selected length
            words_all = " ".join(" ".join(words) for words in words_all[words_all.apply(lambda x: len(x) > 0)]) #Joining all words of the dataframe into one big string
            if words_all:
                wc = WordCloud(background_color="white", max_words=100, mask=words_mask, contour_width=3, contour_color='steelblue') #Initializing WordCloud object
                wc.generate(words_all) #Generating the word cloud into the WordCloud object
                axes[2].imshow(wc, interpolation="bilinear") #Drawing word cloud on axes[2]
            else:
                axes[2].imshow(words_mask, cmap='gray', interpolation="bilinear") #If no words then just show the mask image
            axes[2].axis("off")
            # SCATTER DOT PLOT
            num_words = filtered_data['number_of_words']
            x_jittered = [1]*len(num_words) + np.random.normal(loc=0, scale=0.06, size=len(num_words)) #Creating a displacement on the x axis to display the dots better
            axes[3].scatter(x_jittered, num_words.values, color='blue', alpha=0.7, edgecolors='black', s=100) #Creating the scatter dot plot
            if num_words.empty:
                axes[3].set_yticks([]) #Remove y ticks when empty
            else:
                axes[3].yaxis.set_major_locator(ticker.MaxNLocator(integer=True))    
            axes[3].set_xlabel("") #Remove x labels
            axes[3].set_xticks([]) #Remove x ticks
            axes[3].set_title("Cantidad de palabras en los comentarios en cada curso") #Setting the title of the plot axes[3]
        else:
            #If no data then display a text and clear all plot lines
            axes[0].text(1, 0, 'No data', ha='center', fontsize=30, transform=axes[0].transAxes)
            for ax in axes:
                ax.set_frame_on(False)
                ax.set_xticks([])
                ax.set_yticks([]) 
            
        plt.show()

When we use ipywidgets we have to set up observer functions that will trigger when their values change.

Also I create an instance of the Output class in order to use it inside Draw_plots and to hold all the plot drawings.

In [14]:
#Output widget
output = widgets.Output() 

#Calls Draw_plots passing the current value of all widgets
def On_value_change(change):
    Draw_plots(widget_q1_checkbox.value, widget_q1_slider.value, widget_q2_checkbox.value, widget_q2_slider.value, widget_letter_slider.value)

#Event trigger for the widgets
widget_q1_slider.observe(On_value_change, names="value")
widget_q1_checkbox.observe(On_value_change, names="value")
widget_q2_slider.observe(On_value_change, names="value")
widget_q2_checkbox.observe(On_value_change, names="value")
widget_letter_slider.observe(On_value_change, names="value")

Finally I display the final_box which contains the five widgets and the output which contains the four plots.

I also run draw_plots once to set up the initial dashboard.<br>
Later it will get called again when the widgets change and redraw itself.
<br><br>

### Example of usage:
If both checkboxes are off, the graphs will be presented using the unmodified dataset.<br>
The word cloud and scatter dot graphs will always filter the words they use according to the minimum word length established by the third slider.<br>
If we activate the first checkbox, we will only consider the rows in our dataset where the mean of the responses for that row is closer to the category selected in the first slider.<br>
The bar chart will display with full opacity and will update according to the filtered dataset.<br>
Similarly, the word cloud and scatter dot graphs will update according to the filtered dataset.<br>
If we also activate the second checkbox the dataset will be further filtered according to the category selected in the second slider.

In [15]:
display(final_box, output) #Displaying the widgets and plots
Draw_plots(widget_q1_checkbox.value, widget_q1_slider.value, widget_q2_checkbox.value, widget_q2_slider.value, widget_letter_slider.value) #Ran once

Output()

## Translation
![Translation](assets/translated.png)